# Canvas Widget for IPython Notebook

In [ ]:
from __future__ import print_function, unicode_literals, division, absolute_import

## Canvas Example with HTML and JavaScript

In [4]:
%%html

<html>
    <head></head>
    <body>
        <canvas id='hello_example' style='border: solid black 2px'/>
    </body>
</html>

In [5]:
%%javascript

// URL to elephant image at wikipedia.com.
var url = 'http://goo.gl/m8a4dq'

// Get the canvas element plus corresponding drawing context
var canvas = document.getElementById('hello_example');
var context = canvas.getContext('2d');

// Create a hidden <img> element to manage incoming data.
var img = new Image();

// Add new-data event handler to the hidden <img> element.
img.onload = function () {
    // This function will be called when new image data has finished loading
    // into the <img> element.  This new data will be the source for drawing
    // onto the Canvas.

    // Set canvas geometry.
    canvas.width = img.width
    canvas.style.width = img.width + 'px'

    canvas.height = img.height
    canvas.style.height = img.height + 'px'

    // Draw new image data onto the Canvas.
    context.drawImage(img, 0, 0);
}

// Assign image URL.
img.src = url

<IPython.core.display.Javascript object>

## My new Class: `CanvasImage`

Have a look at the JavaScript source code for the `CanvasWidget` I wrote based on the IPython Notebook's widget infrastructure.

You'll see components similar to the example just above at the core of my widget.  The full version is more complicated due to it being implemented as part of an existing framework that itself is a sophisticated mix of multiple JavaScript libraries (but mostly BackboneJS in this context).

In the next cell I use my new Canvas Image Widget to load and display the same image as above, except this time it's so much simpler.

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore', module='IPython.html.widgets')

import requests
import IPython.html.widgets

import widget_canvas
from widget_canvas import image

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
url_elephant = 'http://upload.wikimedia.org/wikipedia/commons/thumb/3/37/African_Bush_Elephant.jpg/160px-African_Bush_Elephant.jpg'

resp = requests.get(url_elephant)

In [ ]:
d = image.decompress(resp.content)

plt.imshow(d)
plt.show()

In [ ]:
# Same URL as earlier
url_elephant = 'http://upload.wikimedia.org/wikipedia/commons/thumb/3/37/African_Bush_Elephant.jpg/160px-African_Bush_Elephant.jpg'

# Make the widget and display it to the screen.
wid = widget_canvas.CanvasImage(url=url_elephant)
wid.display()

In [ ]:
wid.data.shape

But what about the nice black line around the edge?  No problem!  The IPython built-in Widget's support direct manipulation of CSS properties.  My Canvas Widget is an extension of the IPython `DOMWidget`, so it too can manipulate CSS properties.  The Mozilla Develloper Network has great reference information for [CSS properties](https://developer.mozilla.org/en-US/docs/Web/CSS/Reference).


In [ ]:
# Update widget's image border property.
wid.border_color = 'solid black 2px'

Using my `CanvasImageWidget` is much easier than stepping through all those hoops needed with the `%%html` or `%%javascript` magic functions.  And that's the whole point of the IPython Notebook!  Make it easier to work with data and view the results.

## Display Images from Numpy Arrays

While displaying images by specifying a URL has many uses, I'm mostly interested in displaying technical imagery as part of my everyday data analysis efforts.  This means working with data in Numpy arrays in the Python back-end and somehow transfering images to the JavaScript front-end in a manner compatible with the existing Widget system.  It turns out it's quite easy to compress an image as PNG and embed that data via [Base64](http://en.wikipedia.org/wiki/Base64) encoding into a very large URL string.  This may not be the most efficient or elegant solution, but it was simplest for me to understand and get it working quickly.  

The little helper function below shows how I do this conversion, using `PIL/Pillow` behind the scenes to perform the actual PNG compression.  The input to the helper function is a Numpy array containing an image, and the output is a huge URL string with embedded image data.

In [ ]:
import base64

def encode_src_data(image):
    """
    Compress and encode input image data.
    Format as <img> element src string with embedded data.
    This function uses a PNG compression function from the included module image.py.
    """
    # Compress with PNG.
    data_comp, fmt = canvas.image.compress(image)

    # Encode with base64.
    data_b64 = base64.b64encode(data_comp)

    # Build an Image Element src string with embedded data.
    data_encode = 'data:image/{:s};base64,{:s}'.format(fmt, data_b64)
    
    return data_encode

## Using `CanvasImage` with Embedded Data 

Here is a quick example of what that encoded URL looks like:

In [ ]:
# Load image file using my simple image reader helper function.
data_whippet = canvas.image.read('images/Whippet.jpg')

print('\nImage shape: {}'.format(data_whippet.shape))

# Encode all test images into src 
src_image = encode_src_data(data_whippet)

print('\nURL number of characters: {:d}'.format(len(src_image)))
print('\nURL first 200 characters: "{:s}..."'.format(src_image[:200]))

This large `src` string with the embedded photo may be fed directly into a `CanvasImageWidget`.

In [ ]:
# Create a CanvasWidget using image data embedded in URL string.
wid = canvas.CanvasImageBase()

wid.data_encode = src_image

wid.display()

I wrote my `CanvasImage` class such that it can directly accept Numpy arrays containing image data and incorporates the above functionality as integral components.  This class handles all the details for embedding the compressed image into the Canvas Element.

In [ ]:
# Load another dog image.
data_doberman = canvas.image.read('images/Doberman.jpg')

# Display Numpy image directly.
wid = canvas.CanvasImageBase(data_doberman)
wid.display()

But which dog to choose?  I like them both!

In [ ]:
import time
num_loop = 100
dt = 0.05

t0 = time.time()
for k in range(num_loop):
    wid.image = [data_whippet, data_doberman][k % 2]
    time.sleep(dt)
    
t1 = time.time()

fps = num_loop / (t1 - t0)
print('Rate: {:5.2f} FPS'.format(fps))